<a href="https://colab.research.google.com/github/danlingzhou16/stat390/blob/Danling/Danling_390_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import warnings
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import time as time
warnings.filterwarnings("ignore")

In [2]:
# load dataset
train = pd.read_csv('train_final.csv', parse_dates = ['date'])
test = pd.read_csv('test_final.csv', parse_dates = ['date'])
# remove useless columns
train = train.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
test = test.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])

In [3]:
# find categorical variables
train.select_dtypes(exclude='number')

,date,location_key_x,country_code,day_name,season
0,2020-01-22,US_AK,US,Wednesday,Winter
1,2020-01-23,US_AK,US,Thursday,Winter
2,2020-01-24,US_AK,US,Friday,Winter
3,2020-01-25,US_AK,US,Saturday,Winter
4,2020-01-26,US_AK,US,Sunday,Winter
...,...,...,...,...,...
69908,2021-12-27,AU_WA,AU,Monday,Summer
69909,2021-12-28,AU_WA,AU,Tuesday,Summer
69910,2021-12-29,AU_WA,AU,Wednesday,Summer
69911,2021-12-30,AU_WA,AU,Thursday,Summer


What to do?

Drop `location_key_x` because population is different in every location. `day_name` is already encoded so it can be dropped as well.

Take out the date. Make a separate series for it because I will need it for plotting, but not modeling.

Label encoding the rest categorical variables.

Also, `new_confirmed_mean1`, `new_confirmed_min1` and `new_confirmed_max1` are removed. They just look like the target variable and I have no idea who put this in when doing the feature engineering...


In [26]:
# WHO THE HECK PUT NEW CONFIRM MEAN 1 IN???
print((train.new_confirmed_min1 == train.new_confirmed).sum() == train.shape[0])
print((train.new_confirmed_max1 == train.new_confirmed).sum() == train.shape[0])
print((train.new_confirmed_mean1 == train.new_confirmed).sum() == train.shape[0])

True
True
True


In [27]:
train_no_lockey = train.drop(columns = ['location_key_x', 'date', 'day_name', 'new_confirmed_mean1', 'new_confirmed_max1', 'new_confirmed_min1'])
test_no_lockey = test.drop(columns = ['location_key_x', 'date', 'day_name', 'new_confirmed_mean1', 'new_confirmed_max1', 'new_confirmed_min1'])
train_date = train.date
test_date = test.date

In [28]:
# label encoding
from sklearn.preprocessing import LabelEncoder

label_encoder1 = LabelEncoder() # for country code
label_encoder2 = LabelEncoder() # for season
# country_code label encoding
train_no_lockey['country_code']= label_encoder1.fit_transform(train_no_lockey['country_code'])
test_no_lockey['country_code']= label_encoder1.transform(test_no_lockey['country_code'])
# day of the week -- turns out there is a day_of_week column that has already been encoded
# however, Mon-Sun is from 0-6 and I want it to be 1-7
train_no_lockey['day_of_week']= train_no_lockey['day_of_week'] + 1
test_no_lockey['day_of_week']= test_no_lockey['day_of_week'] + 1
# season
train_no_lockey['season'] = label_encoder2.fit_transform(train_no_lockey['season'])
test_no_lockey['season']= label_encoder2.transform(test_no_lockey['season'])

In [29]:
# prepare for time series split -- order the training dataset
train_num_with_date = pd.concat([train_date, train_no_lockey], axis = 1)
train_by_date = train_num_with_date.sort_values(by = 'date', ascending = True)
train_by_date.reset_index(inplace=True, drop = True)
train_by_date.tail()

,date,country_code,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,...,new_confirmed_min3,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season
69908,2021-12-31,9,2.0,1108.0,733391.0,424916.0,391925.0,64.000000,-150.000000,1717856.0,...,277.0,56.0,5,4,12,2021,365,31,52,3
69909,2021-12-31,6,0.0,0.0,243385.0,120320.0,116315.0,66.833333,14.666667,38154.0,...,0.0,0.0,5,4,12,2021,365,31,52,3
69910,2021-12-31,1,2.0,1604.0,1520968.0,757081.0,763887.0,47.413056,8.656389,1729.0,...,2113.0,1239.0,5,4,12,2021,365,31,52,3
69911,2021-12-31,9,0.0,3066.0,1097379.0,297434.0,335235.0,41.700000,-71.500000,3144.0,...,0.0,0.0,5,4,12,2021,365,31,52,3
69912,2021-12-31,0,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.000000,121.000000,2527013.0,...,0.0,0.0,5,4,12,2021,365,31,52,2


In [30]:
# Seperate X and y!
y_train = train_by_date['new_confirmed']
X_train = train_by_date.drop(columns = ['date', 'new_confirmed'])
y_test = test_no_lockey['new_confirmed']
X_test = test_no_lockey.drop(columns = ['new_confirmed'])

In [31]:
# try a lgbm model
lgbm = LGBMRegressor(random_state=1, n_jobs=-1)
lgbm.fit(X_train, y_train)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012187 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6537
[LightGBM] [Info] Number of data points in the train set: 69913, number of used features: 177
[LightGBM] [Info] Start training from score 825.586314


LGBMRegressor(n_jobs=-1, random_state=1)

In [32]:
feature_importance_table = pd.DataFrame(data = [lgbm.feature_name_, lgbm.feature_importances_]).T
feature_importance_table.columns = ['feature', 'importance']
feature_importance_table.sort_values(by = 'importance', ascending = False)

,feature,importance
163,new_confirmed_lag1,293
1,new_deceased,240
165,new_confirmed_lag7,224
166,new_confirmed_mean3,213
171,new_confirmed_max3,201
...,...,...
71,NY.ADJ.DCO2.GN.ZS,0
72,NY.ADJ.DKAP.CD,0
73,NY.ADJ.DMIN.CD,0
74,NY.ADJ.DMIN.GN.ZS,0


In [33]:
import_feature = feature_importance_table.loc[feature_importance_table.importance != 0].sort_values(by = 'importance', ascending = False)
import_feature.reset_index()

,index,feature,importance
0,163,new_confirmed_lag1,293
1,1,new_deceased,240
2,165,new_confirmed_lag7,224
3,166,new_confirmed_mean3,213
4,171,new_confirmed_max3,201
5,169,new_confirmed_std3,200
6,175,day_of_week,196
7,173,new_confirmed_min3,194
8,164,new_confirmed_lag3,181
9,10,mobility_workplaces,137


In [34]:
# find the rmse of this untuned lgbm model

# train and test rmse
y_train_pred = lgbm.predict(X_train)
y_test_pred = lgbm.predict(X_test)
print("The training rmse is ", mean_squared_error(y_train, y_train_pred, squared = False))
print("The testing rmse is ", mean_squared_error(y_test, y_test_pred, squared = False))

The training rmse is  356.0151974194025
The testing rmse is  3864.7190416233884


See if this model can be improved. Since tuning with so many columns takes forever, I will only use features that are important in the first model.

In [35]:
Sel_features = feature_importance_table.loc[feature_importance_table.importance != 0, 'feature']
X_train_sel = X_train[Sel_features]
X_test_sel = X_test[Sel_features]

In addition, vaccination seems to be an important feature, and before vaccination comes our all the 0's should be treated as outlier. Therefore, I am excluding the dates before 2020/12/11. Date source: https://www.fda.gov/news-events/press-announcements/fda-approves-first-covid-19-vaccine#:~:text=Since%20Dec.,age%20on%20May%2010%2C%202021.

In [36]:
X_train_sel_vac = X_train_sel.loc[train_by_date.date >= '2020-12-11']
y_train_sel_vac = y_train.loc[train_by_date.date >= '2020-12-11']

In [37]:
# try a lgbm model
lgbm2 = LGBMRegressor(random_state=1, n_jobs=-1)
lgbm2.fit(X_train_sel_vac, y_train_sel_vac)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001479 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4997
[LightGBM] [Info] Number of data points in the train set: 38801, number of used features: 32
[LightGBM] [Info] Start training from score 1068.856421


LGBMRegressor(n_jobs=-1, random_state=1)

In [38]:
feature_importance_table2 = pd.DataFrame(data = [lgbm2.feature_name_, lgbm2.feature_importances_]).T
feature_importance_table2.columns = ['feature', 'importance']
feature_importance_table2.sort_values(by = 'importance', ascending = False, inplace = True)
feature_importance_table2.reset_index()

,index,feature,importance
0,14,new_confirmed_lag1,274
1,16,new_confirmed_lag7,262
2,19,new_confirmed_std3,256
3,21,new_confirmed_max3,215
4,17,new_confirmed_mean3,211
5,0,new_deceased,197
6,25,day_of_week,183
7,15,new_confirmed_lag3,174
8,23,new_confirmed_min3,172
9,8,mobility_workplaces,149


In this case `cumulative_persons_fully_vaccinated` and `new_persons_fully_vaccinated` become slightly more important. Let's check rmse.

In [39]:
# train and test rmse
y_train_pred2 = lgbm2.predict(X_train_sel_vac)
y_test_pred2 = lgbm2.predict(X_test_sel)
print("The training rmse is ", mean_squared_error(y_train_sel_vac, y_train_pred2, squared = False))
print("The testing rmse is ", mean_squared_error(y_test, y_test_pred2, squared = False))

The training rmse is  401.3457256518574
The testing rmse is  3539.724981189904


This model performs better on the testing data! This suggest that removing the data before the vaccine came out is a reasonable strategy. Now attempt grid search to tune this model.

In [40]:
# fit a lgbm model

start_time = time.time()
lgbm_grid1 = {'num_leaves': np.arange(21, 42, 10),
             'max_depth': [4, 6, 8],
             'learning_rate': [0.01, 0.1],
             'n_estimators':[50, 100, 200],
              'subsample_for_bin': [500, 1000, 5000],
             'subsample': [0.5, 0.75, 1],
             'reg_alpha': [0, 0.1, 0.5, 1]}
tscv = TimeSeriesSplit(3)

lgbm_gs1 = RandomizedSearchCV(estimator=LGBMRegressor(random_state=1, n_jobs=-1),param_distributions = lgbm_grid1,verbose = True, cv = tscv, n_jobs=-1, scoring = 'neg_root_mean_squared_error', random_state = 1, n_iter = 2)
lgbm_gs1.fit(X_train_sel_vac,y_train_sel_vac)
print("Optimal parameter values =", lgbm_gs1.best_params_)
print("Optimal cross validation RMSE = ", np.abs(lgbm_gs1.best_score_))
print("Time taken = ", round((time.time()-start_time)/60), " minutes")



Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002440 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4769
[LightGBM] [Info] Number of data points in the train set: 38801, number of used features: 32
[LightGBM] [Info] Start training from score 1068.856421
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

In [41]:
# feature importance
best_lgbm1 = lgbm_gs1.best_estimator_
feature_importance_table = pd.DataFrame(data = [best_lgbm1.feature_name_, best_lgbm1.feature_importances_]).T
feature_importance_table.columns = ['feature', 'importance']
feature_importance_table.sort_values(by = 'importance', ascending = False)

,feature,importance
21,new_confirmed_max3,77
14,new_confirmed_lag1,68
17,new_confirmed_mean3,61
16,new_confirmed_lag7,60
23,new_confirmed_min3,57
19,new_confirmed_std3,40
0,new_deceased,38
15,new_confirmed_lag3,35
25,day_of_week,29
8,mobility_workplaces,28


In [42]:
# train and test rmse
y_train_pred3 = best_lgbm1.predict(X_train_sel_vac)
y_test_pred3 = best_lgbm1.predict(X_test_sel)
print("The training rmse is ", mean_squared_error(y_train_sel_vac, y_train_pred3, squared = False))
print("The testing rmse is ", mean_squared_error(y_test, y_test_pred3, squared = False))

The training rmse is  693.6075787768646
The testing rmse is  3683.0712573645033


Tuning is not attempted anymore because a 3-fold, 2-iteration tuning already took 4 min on a 30-core virtual machine and there seems to be no improvement in performance.

In [ ]:
# visualization later